In [1]:
#import packages
import pandas as pd # data manipulation
import numpy as np
from scipy import sparse

# Load Data

In [2]:
#air_visit_data.csv: historical visit data for the air restaurants as main training dataset
train_data = pd.read_csv('../input/air_visit_data.csv')

#store info
air_store_info = pd.read_csv('../input/air_store_info.csv')
hpg_store_info = pd.read_csv('../input/hpg_store_info.csv')

#reservation info
air_reserve = pd.read_csv('../input/air_reserve.csv')
hpg_reserve = pd.read_csv('../input/hpg_reserve.csv')

store_id_relation = pd.read_csv('../input/store_id_relation.csv')
test_data = pd.read_csv('../input/sample_submission.csv')
date_info = pd.read_csv('../input/date_info.csv').rename(columns={'calendar_date':'visit_date'})
train_size = train_data.shape[0]

# Data Preprocessing

### Display and merge train & test data for further feature engineering

In [3]:
display(train_data.head())
display(test_data.head())

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6


,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,0
1,air_00a91d42b08b08d9_2017-04-24,0
2,air_00a91d42b08b08d9_2017-04-25,0
3,air_00a91d42b08b08d9_2017-04-26,0
4,air_00a91d42b08b08d9_2017-04-27,0


In [4]:
#split store id and visit date in test_data id
test_data['visit_date'] = test_data['id'].map(lambda x: x.split('_')[2])
test_data['air_store_id'] = test_data['id'].map(lambda x: '_'.join(x.split('_'[:2])))
#test_data.drop(['id'], axis = 1, inplace = True)

In [5]:
test_data.head()

,id,visitors,visit_date,air_store_id
0,air_00a91d42b08b08d9_2017-04-23,0,2017-04-23,air_00a91d42b08b08d9_2017-04-23
1,air_00a91d42b08b08d9_2017-04-24,0,2017-04-24,air_00a91d42b08b08d9_2017-04-24
2,air_00a91d42b08b08d9_2017-04-25,0,2017-04-25,air_00a91d42b08b08d9_2017-04-25
3,air_00a91d42b08b08d9_2017-04-26,0,2017-04-26,air_00a91d42b08b08d9_2017-04-26
4,air_00a91d42b08b08d9_2017-04-27,0,2017-04-27,air_00a91d42b08b08d9_2017-04-27


In [6]:
#merge train_data and test_data
full_data = pd.concat([train_data, test_data]).reset_index(drop = True)

In [7]:
full_data.head()

,air_store_id,id,visit_date,visitors
0,air_ba937bf13d40fb24,NaN,2016-01-13,25
1,air_ba937bf13d40fb24,NaN,2016-01-14,32
2,air_ba937bf13d40fb24,NaN,2016-01-15,29
3,air_ba937bf13d40fb24,NaN,2016-01-16,22
4,air_ba937bf13d40fb24,NaN,2016-01-18,6


### Exploration of Datetime Features
Get more information about visit time, eg. the day of week/ is month end

In [8]:
#change data type from string to datetime
full_data['visit_date'] = pd.to_datetime(full_data['visit_date'])

In [9]:
#preprocessing datetime data using pandas.Series.dt
full_data['dow'] = full_data['visit_date'].dt.dayofweek
full_data['year'] = full_data['visit_date'].dt.year
full_data['month'] = full_data['visit_date'].dt.month
full_data['doy'] = full_data['visit_date'].dt.dayofyear
full_data['dom'] = full_data['visit_date'].dt.days_in_month
full_data['woy'] = full_data['visit_date'].dt.weekofyear
full_data['is_month_end'] = full_data['visit_date'].dt.is_month_end
full_data['visit_date'] = full_data['visit_date'].dt.date

In [10]:
# change the format from 20160113 to "20160113"
full_data['date_int'] = full_data['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)

### Exploration of Area Features
Split area names in store information (with space)
<p>eg. 'Hyōgo-ken Kōbe-shi Kumoidōri' contains three levels of area info: 1.Hyōgo-ken; 2.Kōbe-shi; 3.Kumoidōri

In [11]:
air_store_info['air_area_lv1'] = air_store_info.air_area_name.apply(lambda x:x.split(' ')[0])
air_store_info['air_area_lv2'] = air_store_info.air_area_name.apply(lambda x:x.split(' ')[1])
air_store_info['air_area_lv3'] = air_store_info.air_area_name.apply(lambda x:x.split(' ')[2])

hpg_store_info['hpg_area_lv1'] = hpg_store_info.hpg_area_name.apply(lambda x:x.split(' ')[0])
hpg_store_info['hpg_area_lv2'] = hpg_store_info.hpg_area_name.apply(lambda x:x.split(' ')[1])
hpg_store_info['hpg_area_lv3'] = hpg_store_info.hpg_area_name.apply(lambda x:x.split(' ')[2])

In [12]:
air_store_info.head()

,air_store_id,air_genre_name,air_area_name,latitude,longitude,air_area_lv1,air_area_lv2,air_area_lv3
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,Hyōgo-ken,Kōbe-shi,Kumoidōri
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,Hyōgo-ken,Kōbe-shi,Kumoidōri
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,Hyōgo-ken,Kōbe-shi,Kumoidōri
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,Hyōgo-ken,Kōbe-shi,Kumoidōri
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,Tōkyō-to,Minato-ku,Shibakōen


Create features that based on different levels of area

In [13]:
#For air_store_info

# Groupby and calculate the number of stores in same area (with different level of areas)
# Named with 'air_stores_on_same_addr' & 'air_stores_lv1' &'air_stores_lv2'
air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby(['latitude','longitude']).air_store_id.count().\
                                    reset_index().rename(columns={'air_store_id':'air_stores_on_same_addr'}),
                             how='left',
                             on=['latitude','longitude'])


air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby('air_area_lv1').air_store_id.count().\
                                    reset_index().rename(columns={'air_store_id':'air_stores_lv1'}),
                             how='left',
                             on='air_area_lv1')

air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby(['air_area_lv1','air_area_lv2']).air_store_id.count().\
                                    reset_index().rename(columns={'air_store_id':'air_stores_lv2'}),
                             how='left',
                             on=['air_area_lv1','air_area_lv2'])


In [14]:
#Calculate the mean/max/min of level-1 latitude
air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby('air_area_lv1').latitude.mean().\
                                    reset_index().rename(columns={'latitude':'mean_lat_air_lv1'}),
                             how='left',
                             on='air_area_lv1')

air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby('air_area_lv1').latitude.max().\
                                    reset_index().rename(columns={'latitude':'max_lat_air_lv1'}),
                             how='left',
                             on='air_area_lv1')

air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby('air_area_lv1').latitude.min().\
                                    reset_index().rename(columns={'latitude':'min_lat_air_lv1'}),
                             how='left',
                             on='air_area_lv1')

#Calculate the mean/max/min of level-1 longitude
air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby('air_area_lv1').longitude.mean().\
                                    reset_index().rename(columns={'longitude':'mean_lon_air_lv1'}),
                             how='left',
                             on='air_area_lv1')

air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby('air_area_lv1').longitude.max().\
                                    reset_index().rename(columns={'longitude':'max_lon_air_lv1'}),
                             how='left',
                             on='air_area_lv1')

air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby('air_area_lv1').longitude.min().\
                                    reset_index().rename(columns={'longitude':'min_lon_air_lv1'}),
                             how='left',
                             on='air_area_lv1')

In [15]:
#Calculate the mean/max/min of level-2 latitude and longitude
air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby(['air_area_lv1','air_area_lv2']).latitude.mean().\
                                    reset_index().rename(columns={'latitude':'mean_lat_air_lv2'}),
                             how='left',
                             on=['air_area_lv1','air_area_lv2'])

air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby(['air_area_lv1','air_area_lv2']).latitude.max().\
                                    reset_index().rename(columns={'latitude':'max_lat_air_lv2'}),
                             how='left',
                             on=['air_area_lv1','air_area_lv2'])

air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby(['air_area_lv1','air_area_lv2']).latitude.min().\
                                    reset_index().rename(columns={'latitude':'min_lat_air_lv2'}),
                             how='left',
                             on=['air_area_lv1','air_area_lv2'])


air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby(['air_area_lv1','air_area_lv2']).longitude.mean().\
                                    reset_index().rename(columns={'longitude':'mean_lon_air_lv2'}),
                             how='left',
                             on=['air_area_lv1','air_area_lv2'])

air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby(['air_area_lv1','air_area_lv2']).longitude.max().\
                                    reset_index().rename(columns={'longitude':'max_lon_air_lv2'}),
                             how='left',
                             on=['air_area_lv1','air_area_lv2'])

air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby(['air_area_lv1','air_area_lv2']).longitude.min().\
                                    reset_index().rename(columns={'longitude':'min_lon_air_lv2'}),
                             how='left',
                             on=['air_area_lv1','air_area_lv2'])

In [16]:
#Same for hpg_store_info

# Groupby and calculate the number of stores in same area (with different level of areas)
# Named with 'air_stores_on_same_addr' & 'air_stores_lv1' &'air_stores_lv2'
hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby(['latitude','longitude']).hpg_store_id.count().\
                                    reset_index().rename(columns={'hpg_store_id':'hpg_stores_on_same_addr'}),
                             how='left',
                             on=['latitude','longitude'])


hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby('hpg_area_lv1').hpg_store_id.count().\
                                    reset_index().rename(columns={'hpg_store_id':'hpg_stores_lv1'}),
                             how='left',
                             on='hpg_area_lv1')

hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby(['hpg_area_lv1','hpg_area_lv2']).hpg_store_id.count().\
                                    reset_index().rename(columns={'hpg_store_id':'hpg_stores_lv2'}),
                             how='left',
                             on=['hpg_area_lv1','hpg_area_lv2'])

#Calculate the mean/max/min of level-1 latitude and longitude
hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby('hpg_area_lv1').latitude.mean().\
                                    reset_index().rename(columns={'latitude':'mean_lat_hpg_lv1'}),
                             how='left',
                             on='hpg_area_lv1')

hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby('hpg_area_lv1').latitude.max().\
                                    reset_index().rename(columns={'latitude':'max_lat_hpg_lv1'}),
                             how='left',
                             on='hpg_area_lv1')

hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby('hpg_area_lv1').latitude.min().\
                                    reset_index().rename(columns={'latitude':'min_lat_hpg_lv1'}),
                             how='left',
                             on='hpg_area_lv1')


hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby('hpg_area_lv1').longitude.mean().\
                                    reset_index().rename(columns={'longitude':'mean_lon_hpg_lv1'}),
                             how='left',
                             on='hpg_area_lv1')

hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby('hpg_area_lv1').longitude.max().\
                                    reset_index().rename(columns={'longitude':'max_lon_hpg_lv1'}),
                             how='left',
                             on='hpg_area_lv1')

hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby('hpg_area_lv1').longitude.min().\
                                    reset_index().rename(columns={'longitude':'min_lon_hpg_lv1'}),
                             how='left',
                             on='hpg_area_lv1')

#Calculate the mean/max/min of level-2 latitude and longitude
hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby(['hpg_area_lv1','hpg_area_lv2']).latitude.mean().\
                                    reset_index().rename(columns={'latitude':'mean_lat_hpg_lv2'}),
                             how='left',
                             on=['hpg_area_lv1','hpg_area_lv2'])

hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby(['hpg_area_lv1','hpg_area_lv2']).latitude.max().\
                                    reset_index().rename(columns={'latitude':'max_lat_hpg_lv2'}),
                             how='left',
                             on=['hpg_area_lv1','hpg_area_lv2'])

hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby(['hpg_area_lv1','hpg_area_lv2']).latitude.min().\
                                    reset_index().rename(columns={'latitude':'min_lat_hpg_lv2'}),
                             how='left',
                             on=['hpg_area_lv1','hpg_area_lv2'])


hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby(['hpg_area_lv1','hpg_area_lv2']).longitude.mean().\
                                    reset_index().rename(columns={'longitude':'mean_lon_hpg_lv2'}),
                             how='left',
                             on=['hpg_area_lv1','hpg_area_lv2'])

hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby(['hpg_area_lv1','hpg_area_lv2']).longitude.max().\
                                    reset_index().rename(columns={'longitude':'max_lon_hpg_lv2'}),
                             how='left',
                             on=['hpg_area_lv1','hpg_area_lv2'])

hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby(['hpg_area_lv1','hpg_area_lv2']).longitude.min().\
                                    reset_index().rename(columns={'longitude':'min_lon_hpg_lv2'}),
                             how='left',
                             on=['hpg_area_lv1','hpg_area_lv2'])

### Merge store information for stores that exist in both air and hpg and add to full_data

In [17]:
air_store_info = pd.merge(air_store_info, store_id_relation, how='left', on='air_store_id')
air_store_info = pd.merge(air_store_info, hpg_store_info, how='left', on='hpg_store_id')
air_store_info = air_store_info.rename(columns={'latitude_x':'latitude_air',
                             'longitude_x':'longitude_air',
                             'latitude_y':'latitude_hpg',
                             'longitude_y':'longitude_hpg'})

In [18]:
air_store_info.head()

,air_store_id,air_genre_name,air_area_name,latitude_air,longitude_air,air_area_lv1,air_area_lv2,air_area_lv3,air_stores_on_same_addr,air_stores_lv1,...,min_lat_hpg_lv1,mean_lon_hpg_lv1,max_lon_hpg_lv1,min_lon_hpg_lv1,mean_lat_hpg_lv2,max_lat_hpg_lv2,min_lat_hpg_lv2,mean_lon_hpg_lv2,max_lon_hpg_lv2,min_lon_hpg_lv2
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,Hyōgo-ken,Kōbe-shi,Kumoidōri,17,57,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,Hyōgo-ken,Kōbe-shi,Kumoidōri,17,57,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,Hyōgo-ken,Kōbe-shi,Kumoidōri,17,57,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,Hyōgo-ken,Kōbe-shi,Kumoidōri,17,57,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,Tōkyō-to,Minato-ku,Shibakōen,51,444,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
#Add store information to full data
full_data = pd.merge(full_data, air_store_info, how='left', on='air_store_id')

In [20]:
full_data.head()

,air_store_id,id,visit_date,visitors,dow,year,month,doy,dom,woy,...,min_lat_hpg_lv1,mean_lon_hpg_lv1,max_lon_hpg_lv1,min_lon_hpg_lv1,mean_lat_hpg_lv2,max_lat_hpg_lv2,min_lat_hpg_lv2,mean_lon_hpg_lv2,max_lon_hpg_lv2,min_lon_hpg_lv2
0,air_ba937bf13d40fb24,NaN,2016-01-13,25,2,2016,1,13,31,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,air_ba937bf13d40fb24,NaN,2016-01-14,32,3,2016,1,14,31,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,air_ba937bf13d40fb24,NaN,2016-01-15,29,4,2016,1,15,31,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,air_ba937bf13d40fb24,NaN,2016-01-16,22,5,2016,1,16,31,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,air_ba937bf13d40fb24,NaN,2016-01-18,6,0,2016,1,18,31,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Reservation Information

In [21]:
print ('before')
display(air_reserve.head())
display(hpg_reserve.head())

before


,air_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5


,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1
1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3
2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2
3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5
4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13


In [22]:
# get the date (without exact time) of visit time/reserve time
air_reserve['visit_date'] = air_reserve['visit_datetime'].apply(lambda x:x[:10])
air_reserve['reserve_date'] = air_reserve['reserve_datetime'].apply(lambda x:x[:10])

In [23]:
# change data type from string to datetime and make some datetime features
air_reserve['visit_datetime'] = pd.to_datetime(air_reserve['visit_datetime'])
air_reserve['visit_date'] = air_reserve['visit_datetime'].dt.date
air_reserve['reserve_datetime'] = pd.to_datetime(air_reserve['reserve_datetime'])
air_reserve['reserve_date'] = air_reserve['reserve_datetime'].dt.date

In [24]:
#??? meaning of reserve_datetime_diff
air_reserve['reserve_datetime_diff'] = air_reserve.apply(lambda r: (r['visit_datetime'] - r['reserve_datetime']).\
                                                         seconds * r['reserve_visitors']/3600/24.0, axis=1)

In [25]:
hpg_reserve.head()

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1
1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3
2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2
3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5
4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13


In [26]:
# Same process as air_reserve
hpg_reserve['visit_date'] = hpg_reserve['visit_datetime'].apply(lambda x:x[:10])
hpg_reserve['reserve_date'] = hpg_reserve['reserve_datetime'].apply(lambda x:x[:10])

In [29]:
hpg_reserve['reserve_datetime'] = pd.to_datetime(hpg_reserve['reserve_datetime'])
hpg_reserve['reserve_date'] = hpg_reserve['reserve_datetime'].dt.date
hpg_reserve['visit_datetime'] = pd.to_datetime(hpg_reserve['visit_datetime'])
hpg_reserve['visit_date'] = hpg_reserve['visit_datetime'].dt.date

In [30]:
hpg_reserve['reserve_datetime_diff'] = hpg_reserve.apply(lambda r: (r['visit_datetime'] - r['reserve_datetime']).seconds
                                                         * r['reserve_visitors']/3600/24.0, axis=1)

In [31]:
print ('after')
display(air_reserve.head())
display(hpg_reserve.head())

after


,air_store_id,visit_datetime,reserve_datetime,reserve_visitors,visit_date,reserve_date,reserve_datetime_diff
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1,2016-01-01,2016-01-01,0.125000
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3,2016-01-01,2016-01-01,0.000000
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6,2016-01-01,2016-01-01,0.000000
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2,2016-01-01,2016-01-01,0.333333
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5,2016-01-01,2016-01-01,3.958333


,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors,visit_date,reserve_date,reserve_datetime_diff
0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1,2016-01-01,2016-01-01,0.083333
1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3,2016-01-01,2016-01-01,0.875000
2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2,2016-01-01,2016-01-01,0.166667
3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5,2016-01-01,2016-01-01,1.250000
4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13,2016-01-01,2016-01-01,7.583333


In [36]:
air_reserve_grp = air_reserve.groupby(['air_store_id','visit_date'])['reserve_visitors','reserve_datetime_diff'].\
            sum().reset_index().rename(columns={'reserve_visitors':'air_rvs',
                                               'reserve_datetime_diff':'air_rv_dt_diff'})
hpg_reserve_grp = hpg_reserve.groupby(['hpg_store_id','visit_date'])['reserve_visitors','reserve_datetime_diff'].\
            sum().reset_index().rename(columns={'reserve_visitors':'hpg_rvs',
                                               'reserve_datetime_diff':'hpg_rv_dt_diff'})
air_reserve_grp['mean_air_rv_dt_diff'] = air_reserve_grp.air_rv_dt_diff / air_reserve_grp.air_rvs
hpg_reserve_grp['mean_hpg_rv_dt_diff'] = hpg_reserve_grp.hpg_rv_dt_diff / hpg_reserve_grp.hpg_rvs    
# ???meaning of mean_hpg_rv_dt_diff?
display(air_reserve_grp.head())
display(hpg_reserve_grp.head())    

,air_store_id,visit_date,air_rvs,air_rv_dt_diff,mean_air_rv_dt_diff
0,air_00a91d42b08b08d9,2016-10-31,2,0.333333,0.166667
1,air_00a91d42b08b08d9,2016-12-05,9,1.500000,0.166667
2,air_00a91d42b08b08d9,2016-12-14,18,6.750000,0.375000
3,air_00a91d42b08b08d9,2016-12-17,2,0.250000,0.125000
4,air_00a91d42b08b08d9,2016-12-20,4,0.500000,0.125000


,hpg_store_id,visit_date,hpg_rvs,hpg_rv_dt_diff,mean_hpg_rv_dt_diff
0,hpg_001112ef76b9802c,2016-02-26,9,2.250000,0.250000
1,hpg_001112ef76b9802c,2016-03-17,3,0.000000,0.000000
2,hpg_001112ef76b9802c,2016-03-31,5,4.791667,0.958333
3,hpg_001112ef76b9802c,2016-04-05,13,4.333333,0.333333
4,hpg_001112ef76b9802c,2016-04-18,9,1.875000,0.208333


In [39]:
# Merge reservation data to full_data
full_data = pd.merge(full_data, air_reserve_grp, how = 'left', on = ['air_store_id', 'visit_date'])
full_data = pd.merge(full_data, hpg_reserve_grp, how = 'left', on = ['hpg_store_id', 'visit_date'])

In [41]:
full_data.query('air_rvs>0 and hpg_rvs>0').head()

,air_store_id,id,visit_date,visitors,dow,year,month,doy,dom,woy,...,min_lat_hpg_lv2,mean_lon_hpg_lv2,max_lon_hpg_lv2,min_lon_hpg_lv2,air_rvs,air_rv_dt_diff,mean_air_rv_dt_diff,hpg_rvs,hpg_rv_dt_diff,mean_hpg_rv_dt_diff
4669,air_3e93f3c81008696d,NaN,2016-10-27,48,3,2016,10,301,31,43,...,NaN,NaN,NaN,NaN,12.0,7.083333,0.590278,2.0,1.750000,0.875000
4670,air_3e93f3c81008696d,NaN,2016-10-28,25,4,2016,10,302,31,43,...,NaN,NaN,NaN,NaN,4.0,1.833333,0.458333,2.0,1.833333,0.916667
4671,air_3e93f3c81008696d,NaN,2016-10-29,42,5,2016,10,303,31,43,...,NaN,NaN,NaN,NaN,17.0,13.000000,0.764706,28.0,18.416667,0.657738
4673,air_3e93f3c81008696d,NaN,2016-11-01,76,1,2016,11,306,30,44,...,NaN,NaN,NaN,NaN,74.0,66.250000,0.895270,3.0,2.625000,0.875000
4676,air_3e93f3c81008696d,NaN,2016-11-04,5,4,2016,11,309,30,44,...,NaN,NaN,NaN,NaN,11.0,4.625000,0.420455,7.0,2.625000,0.375000


## Date Information

In [42]:
# Add datetime features in date_info data
date_info['visit_date'] = pd.to_datetime(date_info['visit_date'])
date_info['dow'] = date_info['visit_date'].dt.dayofweek
date_info['date_len'] = len(date_info)
date_info['date_index'] = date_info.index + 1
date_info['weight'] = ((date_info.index + 1) / len(date_info)) ** 5  #??? weight?
date_info['visit_date'] = date_info['visit_date'].dt.date
date_info.head()

,visit_date,day_of_week,holiday_flg,dow,date_len,date_index,weight
0,2016-01-01,Friday,1,4,517,1,2.707368e-14
1,2016-01-02,Saturday,1,5,517,2,8.663577e-13
2,2016-01-03,Sunday,1,6,517,3,6.578904e-12
3,2016-01-04,Monday,0,0,517,4,2.772345e-11
4,2016-01-05,Tuesday,0,1,517,5,8.460525e-11


In [43]:
full_data = pd.merge(full_data, date_info[['visit_date','date_len','date_index','weight', 'holiday_flg']], 
                     how='left', on='visit_date')

In [45]:
full_data.head()

,air_store_id,id,visit_date,visitors,dow,year,month,doy,dom,woy,...,air_rvs,air_rv_dt_diff,mean_air_rv_dt_diff,hpg_rvs,hpg_rv_dt_diff,mean_hpg_rv_dt_diff,date_len,date_index,weight,holiday_flg
0,air_ba937bf13d40fb24,NaN,2016-01-13,25,2,2016,1,13,31,2,...,NaN,NaN,NaN,NaN,NaN,NaN,517,13,1.005227e-08,0
1,air_ba937bf13d40fb24,NaN,2016-01-14,32,3,2016,1,14,31,2,...,NaN,NaN,NaN,NaN,NaN,NaN,517,14,1.456087e-08,0
2,air_ba937bf13d40fb24,NaN,2016-01-15,29,4,2016,1,15,31,2,...,NaN,NaN,NaN,NaN,NaN,NaN,517,15,2.055908e-08,0
3,air_ba937bf13d40fb24,NaN,2016-01-16,22,5,2016,1,16,31,2,...,NaN,NaN,NaN,NaN,NaN,NaN,517,16,2.838881e-08,0
4,air_ba937bf13d40fb24,NaN,2016-01-18,6,0,2016,1,18,31,3,...,NaN,NaN,NaN,NaN,NaN,NaN,517,18,5.115756e-08,0


In [47]:
# Save to csv file
full_data.to_csv('../input/full_data.csv', index = False)